In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = "/content/drive/MyDrive/data"

In [ ]:
root_path = root_path + "/"

In [ ]:
#installing required libraries
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install bert_score
!pip install scikit-learn

In [ ]:
!pip install -U pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install -U seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from datasets import ClassLabel, Value
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import collections
from tqdm.auto import tqdm
import numpy as np
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from evaluate import load
import evaluate
import json
import torch
from torch.utils.data import Dataset, DataLoader


/soe/vigneshs/miniconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
device

device(type='cuda', index=2)

In [ ]:
root_path = root_path+"qa/"
model_results = root_path+"results"

original_train_path = root_path+"train_qa_data_all.json"
original_val_path = root_path+"val_qa_data_all.json"
original_val_top10 = root_path+"val_qa_data_custom_bert_lstm_top_10.json"
original_val_top4 = root_path+"val_qa_data_custom_bert_lstm_top_4.json"
original_val_oracle = root_path+"val_qa_data_oracle.json"

original_train_path_reformat = root_path+"train_qa_data_all_reformat.json"
original_val_path_reformat = root_path+"val_qa_data_all_reformat.json"
original_val_top10_reformat = root_path+"val_qa_data_custom_bert_lstm_top_10_reformat.json"
original_val_top4_reformat = root_path+"val_qa_data_custom_bert_lstm_top_4_reformat.json"
original_val_oracle_reformat = root_path+"val_qa_data_oracle_reformat.json"

original_train_path_unstructured = root_path+"train_qa_data_all_unstructured.json"
original_val_path_unstructured = root_path+"val_qa_data_all_unstructured.json"
original_val_top10_unstructured = root_path+"val_qa_data_custom_bert_lstm_top_10_unstructured.json"
original_val_top4_unstructured = root_path+"val_qa_data_custom_bert_lstm_top_4_unstructured.json"
original_val_oracle_unstructured = root_path+"val_qa_data_oracle_unstructured.json"

original_train_path_structured = root_path+"train_qa_data_all_structured.json"
original_val_path_structured = root_path+"val_qa_data_all_structured.json"
original_val_top10_structured = root_path+"val_qa_data_custom_bert_lstm_top_10_structured.json"
original_val_top4_structured = root_path+"val_qa_data_custom_bert_lstm_top_4_structured.json"
original_val_oracle_structured = root_path+"val_qa_data_oracle_structured.json"

In [ ]:
#loading model check points for tokenizers
bleu = evaluate.load("bleu")
# model_checkpoint = "deepset/bert-base-uncased-squad2"
model_checkpoint = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
bertscore = load("bertscore")
model_type_bert_score = "distilbert-base-uncased"

In [ ]:
# model_state_dict = torch.load(model_path)
# model.load_state_dict(model_state_dict)
# model = model.to(device)

In [ ]:

def reformat_write(source_p, destination_p):
  f = json.load(open(source_p,"r"))
  json_object_ = json.dumps(f)
  with open(destination_p, "w") as outfile:
    outfile.write(json_object_)


In [ ]:
reformat_write(original_train_path, original_train_path_reformat)
reformat_write(original_val_top10, original_val_top10_reformat)
reformat_write(original_val_path , original_val_path_reformat)
reformat_write(original_val_top4, original_val_top4_reformat)
reformat_write(original_val_oracle, original_val_oracle_reformat)


In [ ]:
def load_data(file_path):
  """
  Reads the input file and extracts sentences for each input data point
  from the answer index and return the list of json objects
  

  Parameters
  ----------
  file_path : Str
      path to the input jsonl file

  Returns
  -------
  filtered_sentences : List
      Contains json objects with the following fields:
      'uuid', 'postText', 'targetTitle', 'targetParagraph', 'spoilerType', 'spoilerPos', 'answers', 'id'

  """

  with open(file_path, 'r') as f:
    sentences = [json.loads(sentence.rstrip()) for sentence in f.readlines()]
    filtered_sentences = []
    c=0
    for _sentence in sentences[0]:
      # print(_sentence)
      # break
      temp_dict = dict()
      for _key, _value in _sentence.items():
        temp_dict[_key] = _value
      temp_dict_ansers = dict()
      temp_text = []
      temp_answer_start = []
      spoilerPoss = temp_dict['spoilerPos']
      if spoilerPoss is not None:
        for spoilpos in spoilerPoss:
          ttext = temp_dict['targetParagraph']
          temp_text.append(ttext[spoilpos[0]:spoilpos[1]])
          temp_answer_start.append(spoilpos[0])
      else:
        temp_text.append("")
        temp_answer_start.append(0)
      temp_dict_ansers['text'] = temp_text
      temp_dict_ansers['answer_start'] = temp_answer_start
      temp_dict['answers'] = temp_dict_ansers
      temp_dict['id'] = c
      c = c+1
      filtered_sentences.append(temp_dict)
      # print(temp_dict)
      


    # filtered_sentences = [{_key: _value for _key, _value in _sentence.items() } for _sentence in sentences[0]]

    return filtered_sentences

In [ ]:
train_unstructured = load_data(original_train_path_reformat)
val_unstructured = load_data(original_val_path_reformat)
val_oracle_unstructured = load_data(original_val_oracle_reformat)

In [ ]:
#remove improper datapoints
train_file = []
for train_file_ in  train_unstructured:
  if train_file_['id']> 30000 and train_file_['id']< 40000:
    continue
  if train_file_['id']> 15000 and train_file_['id']< 17000:
    continue
  # if train_file_['id']> 46998 and train_file_['id']< 47997:
  #   continue
  else:
    train_file.append(train_file_)
val_file = []
for val_file_ in  val_unstructured:
  if val_file_['id']> 1000 and val_file_['id']< 2000:
    continue
  # if val_file_['id']> 2000 and val_file_['id']< 3000:
  #   continue
  else:
    val_file.append(val_file_)

In [ ]:
def write_structured_data(structured_data, out_file_path):
    """
    Writes the strucured data on the disk in json format

    Parameters
    ----------
    structured_data : Dict
        Contains data to be written
    out_file_path : Str
        Path of the file

    Returns
    -------
    None.

    """
    with open(out_file_path, 'w') as outfile:
        for entry in structured_data:
            json.dump(entry, outfile)
            outfile.write('\n')

In [ ]:
# train_file = train_file[:50]
write_structured_data(train_file, original_train_path_structured)

In [ ]:
# val_file = val_file[:50]
write_structured_data(val_file, original_val_path_structured)
write_structured_data(val_oracle_unstructured, original_val_oracle_structured)

In [ ]:

train_file[0]

{'uuid': '0af11f6b-c889-4520-9372-66ba25cb7657',
 'targetParagraphId': 0,
 'postText': 'Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea',
 'targetTitle': 'Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea',
 'targetParagraph': 'Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea',
 'spoilerType': None,
 'spoilerPos': None,
 'answers': {'text': [''], 'answer_start': [0]},
 'id': 0}

In [ ]:
#reading structured to Hugging face dataset object
dict_train = load_dataset("json", data_files= original_train_path_structured , split='train')
dict_val = load_dataset("json", data_files= original_val_path_structured , split='train')
dict_val_oracle = load_dataset("json", data_files= original_val_oracle_structured, split='train')

Using custom data configuration default-3ba48e9225bcc652


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 357.63it/s]


Dataset json downloaded and prepared to /soe/vigneshs/.cache/huggingface/datasets/json/default-3ba48e9225bcc652/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Using custom data configuration default-c65eb360fe2945a3


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 357.45it/s]


Dataset json downloaded and prepared to /soe/vigneshs/.cache/huggingface/datasets/json/default-c65eb360fe2945a3/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Using custom data configuration default-e09b0a47ffdc56a1


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 415.77it/s]


Dataset json downloaded and prepared to /soe/vigneshs/.cache/huggingface/datasets/json/default-e09b0a47ffdc56a1/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


In [ ]:
#renaming and dropping columns in training file
dict_train = dict_train.rename_column("targetTitle", "title")
dict_train = dict_train.rename_column("targetParagraph", "context")
dict_train = dict_train.rename_column("postText", "question")

dict_train = dict_train.remove_columns(['spoilerPos'])

In [ ]:
#renaming and dropping columns in validation file
dict_val = dict_val.rename_column("targetTitle", "title")
dict_val = dict_val.rename_column("targetParagraph", "context")
dict_val = dict_val.rename_column("postText", "question")

dict_val = dict_val.remove_columns(['spoilerPos'])

In [ ]:
#renaming and dropping columns in validation file oracle
dict_val_oracle = dict_val_oracle.rename_column("targetTitle", "title")
dict_val_oracle = dict_val_oracle.rename_column("targetParagraph", "context")
dict_val_oracle = dict_val_oracle.rename_column("postText", "question")

dict_val_oracle = dict_val_oracle.remove_columns(['spoilerPos'])

In [ ]:
dict_train

Dataset({
    features: ['uuid', 'targetParagraphId', 'question', 'title', 'context', 'spoilerType', 'answers', 'id'],
    num_rows: 36563
})

In [ ]:
#initializing tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# Tokenize the training data
def preprocess_function(examples):
      """
      Tokenizes the input data and also formats it according to the input format of bert trained on squad.
      
      Parameters
      ----------
      examples : Dataset Dict

      Returns
      -------
      inputs : Dataset Dict

      """
  # try:
      questions = [q.strip() for q in examples["question"]]
      inputs = tokenizer(
          questions,
          examples["context"],
          max_length=384,
          truncation="only_second",
          return_offsets_mapping=True,
          padding="max_length",
      )

      offset_mapping = inputs.pop("offset_mapping")
      answers = examples["answers"]
      start_positions = []
      end_positions = []

      try:
        for i, offset in enumerate(offset_mapping):
            answer = answers[i]
            start_char = answer["answer_start"][0]
            try:
              end_char = answer["answer_start"][0] + len(answer["text"][0])
            except Exception as E:

              end_char = 0 + len(answer["text"][0])

            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0

            while sequence_ids[idx] != 1:
                idx += 1

            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label it (0, 0)
            try:
              if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                  start_positions.append(0)
                  end_positions.append(0)
              else:
                  # Otherwise it's the start and end token positions
                  idx = context_start
                  while idx <= context_end and offset[idx][0] <= start_char:
                      idx += 1
                  start_positions.append(idx - 1)

                  idx = context_end
                  while idx >= context_start and offset[idx][1] >= end_char:
                      idx -= 1
                  end_positions.append(idx + 1)
            except:
              start_positions.append(0)
              end_positions.append(0)
      except:
        print(examples["id"])

        

      inputs["start_positions"] = start_positions
      inputs["end_positions"] = end_positions
      return inputs


tokenized_train = dict_train.map(preprocess_function, batched=True)#, remove_columns=dict_train["train"].column_names)
tokenized_val = dict_val.map(preprocess_function, batched=True)#, remove_columns=dict_train["train"].column_names)

 92%|█████████▏| 11/12 [00:01<00:00,  5.75ba/s]


In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
#Training the model
training_args = TrainingArguments(
    output_dir=model_results,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()



The following columns in the training set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: uuid, spoilerType, title, id, answers, targetParagraphId, question, context. If uuid, spoilerType, title, id, answers, targetParagraphId, question, context are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
/soe/vigneshs/miniconda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 36563
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2286
  Number of trainable parameters 

Epoch,Training Loss,Validation Loss
1,0.278400,0.311115


Saving model checkpoint to /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-500
Configuration saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-500/config.json
Model weights saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-1000
Configuration saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-1000/config.json
Model weights saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /soe/vigneshs/projects/nlp_243/project_1/data/qa_1/results/che

TrainOutput(global_step=2286, training_loss=0.37177214328580954, metrics={'train_runtime': 736.282, 'train_samples_per_second': 49.659, 'train_steps_per_second': 3.105, 'total_flos': 7165344987403776.0, 'train_loss': 0.37177214328580954, 'epoch': 1.0})

In [ ]:
#tokenize the validation data and Oracle data
def preprocess_function_val(examples):
    """
    Tokenizes the validation data and also formats it according to the input format of bert trained on squad.
    
    Parameters
    ----------
    examples : Dataset Dict

    Returns
    -------
    inputs : Dataset Dict

    """
  # try:
    questions = [q.strip() for q in examples["question"]]
    try:
      inputs = tokenizer(
          questions,
          examples["context"],
          max_length=384,
          truncation="only_second",
          return_offsets_mapping=True,
          return_overflowing_tokens=True,
          padding="max_length",
      )
    except Exception as E:
      # print(e)
      pass
      # print(examples['id'])


    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    max_len = min(len(inputs['offset_mapping']),len(inputs['input_ids']),len(inputs['attention_mask']), 1000 )
    # Some Lenghts are problematic and have been handled here
    print(max_len)
    if(max_len < 1000):
      max_len = min(len(inputs['offset_mapping']),len(inputs['input_ids']),len(inputs['attention_mask']))
      if(max_len < 790):
        max_len = 400
    if len(inputs['input_ids']) == 824:
      max_len = 818 
    if len(inputs['input_ids']) == 407:
      max_len = 406
      # print(len(inputs['offset_mapping']),len(inputs['input_ids']),len(inputs['attention_mask']))
    inputs['input_ids'] = inputs['input_ids'][:max_len]
    # inputs['token_type_ids'] = inputs['token_type_ids'][:max_len]
    inputs['attention_mask'] = inputs['attention_mask'][:max_len]
    inputs['offset_mapping'] = inputs['offset_mapping'][:max_len]

    try:
      for i in range(len(inputs["input_ids"])):
          sample_idx = sample_map[i]
          example_ids.append(examples["id"][sample_idx])

          sequence_ids = inputs.sequence_ids(i)
          offset = inputs["offset_mapping"][i]
          inputs["offset_mapping"][i] = [
              o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
          ]

      inputs["example_id"] = example_ids
    except:
      pass
      # print("a")
      # print(examples['id'])

    return inputs

In [ ]:
tokenized_val = dict_val.map(preprocess_function_val, batched=True)
tokenized_val_oracle = dict_val_oracle.map(preprocess_function_val, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

1000


 17%|█▋        | 2/12 [00:00<00:03,  2.93ba/s]

1000


 25%|██▌       | 3/12 [00:00<00:02,  3.40ba/s]

...

 33%|███▎      | 1/3 [00:00<00:00,  2.11ba/s]

1000


 67%|██████▋   | 2/3 [00:00<00:00,  3.20ba/s]

1000
824


 67%|██████▋   | 2/3 [00:00<00:00,  2.36ba/s]


In [ ]:


def compute_metrics(start_logits, end_logits, features, examples):
    """
    Converts the logits back to text and returns the predicted text and the original text
    """
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    n_best = 20
    max_answer_length = 30
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return predicted_answers, theoretical_answers, examples
    # return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
#######


# LOAD THE MODEL

#########



In [ ]:
training_args = TrainingArguments(
    output_dir=model_results,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#Restructure the predictions and print the blue score for complete validation data and oracle data
def print_single_blue(predicted_answers, theoretical_answers):
  pred_list = []
  references_list = []
  for i in range(0,len(predicted_answers) ):
    pred_list.append(predicted_answers[i]['prediction_text'])
    references_list.append(theoretical_answers[i]['answers']['text'])
  results = bleu.compute(predictions=pred_list, references=references_list)
  print(results)
  return

In [ ]:

#Restructure the predictions and print the bert score for complete validation data and oracle datadef print_single_bert(predicted_answers, theoretical_answers):
def print_single_bert(predicted_answers, theoretical_answers):
  pred_list = []
  references_list = []
  for i in range(0,len(predicted_answers) ):
    pred_list.append(predicted_answers[i]['prediction_text'])
    references_list.append(theoretical_answers[i]['answers']['text'])
  results = bertscore.compute(predictions=pred_list, references=references_list, model_type=model_type_bert_score)
  # print(results)
  print("F1 Score")
  print(sum(results['f1'])/len(results['f1']))

  print("Precision Score")
  print(sum(results['precision'])/len(results['precision']))

  print("Recall Score")
  print(sum(results['recall'])/len(results['recall']))
  return

In [ ]:
#get Predictions on the complete validation data and print bert and bleu scores
predictions, _, _ = trainer.predict(tokenized_val)
start_logits, end_logits = predictions
predicted_answers, theoretical_answers, examples = compute_metrics(start_logits, end_logits, tokenized_val, dict_val)
print("Bleu Score For Entire Validation Set")
print_single_blue(predicted_answers, theoretical_answers)
print_single_bert(predicted_answers, theoretical_answers)


The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping. If uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 11406
  Batch size = 16


100%|██████████| 11406/11406 [00:06<00:00, 1668.85it/s]


Bleu Score For Entire Validation Set
{'bleu': 0.4503579329095682, 'precisions': [0.14760763813257824, 0.6523749474569147, 0.6541772151898734, 0.6530249110320284], 'brevity_penalty': 1.0, 'length_ratio': 1.133020730503455, 'translation_length': 13773, 'reference_length': 12156}


F1 Score
0.0668725916534772
Precision Score
0.07005721556655403
Recall Score
0.0644052673144945


In [ ]:
#get Predictions on the oracle data and print bert and bleu scores
predictions, _, _ = trainer.predict(tokenized_val_oracle)
start_logits, end_logits = predictions
predicted_answers, theoretical_answers, examples = compute_metrics(start_logits, end_logits, tokenized_val_oracle, dict_val_oracle)
print("Bleu Score For Oracle Validation Set")
print_single_bert(predicted_answers, theoretical_answers)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping. If uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2818
  Batch size = 16


100%|██████████| 2818/2818 [00:01<00:00, 1701.05it/s]


Bleu Score For Oracle Validation Set


F1 Score
0.7646873328620614
Precision Score
0.7990863014930017
Recall Score
0.7384735752370359


Getting Results on Top 10 and Top 4 Predictios of the Classification Model

In [ ]:
def load_data_top(file_path):
    """
    Reads the input file and extracts sentences for each input data point
    from the answer index and return the list of json objects
    

    Parameters
    ----------
    file_path : Str
        path to the input jsonl file

    Returns
    -------
    filtered_sentences : List
        Contains json objects with the following fields:
        'uuid', 'postText', 'targetTitle', 'targetParagraph', 'spoilerType', 'spoilerPos', 'answers', 'id'

    """

    with open(file_path, 'r') as f:
      sentences = [json.loads(sentence.rstrip()) for sentence in f.readlines()]
      filtered_sentences = []
      c=0
      for _sentence in sentences[0]:

        temp_dict = dict()
        for _key, _value in _sentence.items():
          temp_dict[_key] = _value
        temp_dict_ansers = dict()
        temp_text = []
        temp_answer_start = []
        spoilerPoss = temp_dict['spoilerPos']
        if spoilerPoss is not None:
          for spoilpos in spoilerPoss:
            ttext = temp_dict['targetParagraph']
            temp_text.append(ttext[spoilpos[0]:spoilpos[1]])
            temp_answer_start.append(spoilpos[0])
        else:
          temp_text.append("")
          temp_answer_start.append(0)
        temp_dict_ansers['text'] = temp_text
        temp_dict_ansers['answer_start'] = temp_answer_start
        temp_dict['answers'] = temp_dict_ansers
        temp_dict['id'] = c
        temp_dict['uuid'] = temp_dict['uuid']
        c = c+1
        filtered_sentences.append(temp_dict)
      return filtered_sentences

In [ ]:
#loading data for top 10 and top4 predictions
val_top10_unstructured = load_data_top(original_val_top10_reformat)
val_top4_unstructured = load_data_top(original_val_top4_reformat)

In [ ]:
#writing structered data to the disk
write_structured_data(val_top10_unstructured, original_val_top10_structured)
write_structured_data(val_top4_unstructured, original_val_top4_structured)

In [ ]:
#Reading Structred data files in a hugging face data object
dict_val_top10 = load_dataset("json", data_files= original_val_top10_structured , split='train')
dict_val_top4 = load_dataset("json", data_files= original_val_top4_structured , split='train')

Using custom data configuration default-1a4125993f594746


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 689.74it/s]


Dataset json downloaded and prepared to /soe/vigneshs/.cache/huggingface/datasets/json/default-1a4125993f594746/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Using custom data configuration default-279a62932ece1b8f


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 852.33it/s]


Dataset json downloaded and prepared to /soe/vigneshs/.cache/huggingface/datasets/json/default-279a62932ece1b8f/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


In [ ]:
#renaming and dropping columns
dict_val_top10 = dict_val_top10.rename_column("targetTitle", "title")
dict_val_top10 = dict_val_top10.rename_column("targetParagraph", "context")
dict_val_top10 = dict_val_top10.rename_column("postText", "question")
dict_val_top10 = dict_val_top10.remove_columns(['spoilerPos'])

In [ ]:
#renaming and dropping columns
dict_val_top4 = dict_val_top4.rename_column("targetTitle", "title")
dict_val_top4 = dict_val_top4.rename_column("targetParagraph", "context")
dict_val_top4 = dict_val_top4.rename_column("postText", "question")

dict_val_top4 = dict_val_top4.remove_columns(['spoilerPos'])

In [ ]:
# Create Tokenizer for Top 10 and Top 4 Data inputs. There are seperate tokenizers for different data sets because of the extra parameter requirements.
def preprocess_function_inference_top(examples):
    """
    Tokenizes the input data and also formats it according to the input format of bert trained on squad.
    
    Parameters
    ----------
    examples : Dataset Dict

    Returns
    -------
    inputs : Dataset Dict

    """
  # try:
    questions = [q.strip() for q in examples["question"]]
    try:
      inputs = tokenizer(
          questions,
          examples["context"],
          max_length=384,
          truncation="only_second",
          return_offsets_mapping=True,
          return_overflowing_tokens=True,
          padding="max_length",
      )
    except:
      pass
      # print(examples['id'])


    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    max_len = min(len(inputs['offset_mapping']),len(inputs['input_ids']),len(inputs['attention_mask']), 1000 )
    # if max_len 
    print(max_len)
    # Some Lenghts are problematic and have been handled here
    if(max_len < 1000):
      max_len = 790
    if len(inputs['input_ids'])== 822:
      max_len = 820
    if len(inputs['input_ids'])== 148:
      max_len = 147
      # print(len(inputs['offset_mapping']),len(inputs['input_ids']),len(inputs['attention_mask']))
    inputs['input_ids'] = inputs['input_ids'][:max_len]
    # inputs['token_type_ids'] = inputs['token_type_ids'][:max_len]
    inputs['attention_mask'] = inputs['attention_mask'][:max_len]
    inputs['offset_mapping'] = inputs['offset_mapping'][:max_len]

    try:
      for i in range(len(inputs["input_ids"])):
          sample_idx = sample_map[i]
          example_ids.append(examples["id"][sample_idx])

          sequence_ids = inputs.sequence_ids(i)
          offset = inputs["offset_mapping"][i]
          inputs["offset_mapping"][i] = [
              o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
          ]

      inputs["example_id"] = example_ids
    except:
      pass
      # print("a")
      # print(examples['id'])

    return inputs



tokenized_val_top10 = dict_val_top10.map(preprocess_function_inference_top, batched=True)
tokenized_val_top4 = dict_val_top4.map(preprocess_function_inference_top, batched=True)

 14%|█▍        | 1/7 [00:00<00:01,  3.33ba/s]

1000


 29%|██▊       | 2/7 [00:00<00:01,  4.10ba/s]

1000
1000


 57%|█████▋    | 4/7 [00:00<00:00,  4.58ba/s]

...

1000


 50%|█████     | 2/4 [00:00<00:00,  3.15ba/s]

1000


 75%|███████▌  | 3/4 [00:00<00:00,  3.22ba/s]

1000
148


In [ ]:
#create tokenizedval_example_id to uuid mapping:
def create_tokenizedval_example_id(tokenized_val_c):
    """
    Create a mapping of index id to uuid which is unique
    for every datapoint in the original.

    Parameters
    ----------
    tokenized_val_c : Dict
        tokenized input

    Returns
    -------
    id_uuid_mapping : Dcit
        Contains a mapping of example_id to uuid

    """
    id_uuid_mapping = dict()
    for i in tokenized_val_c:
      temp_uid = i['uuid']
      id_uuid_mapping[i['example_id']] = temp_uid
      # id_uuid_mapping[temp_uid] = id_uuid_mapping.get(temp_uid, [])+[i['example_id']]
    return id_uuid_mapping 



In [ ]:
#getting id and uuid mappings for both the inputs
id_uuid_mapping_top10 = create_tokenizedval_example_id(tokenized_val_top10)
id_uuid_mapping_top4 = create_tokenizedval_example_id(tokenized_val_top4)

In [ ]:
# Bleu Score
def print_bleu_score_top(uuid_theory,uuid_pred):
    """
    Reformats the data according to the input format of bleu.
    Calculates the bleu score and prints it.

    Parameters
    ----------
    uuid_theory : Dict
        Contains the original labels
    uuid_pred : Dict
        Contains the answers predicted by the model

    Returns
    -------
    None.

    """
    final_predictions_list = []
    final_reference_list = []

    for key, value in uuid_theory.items():
      temp_ref_list= value
      temp_pred_list = uuid_pred[key]
      for i in temp_pred_list:
        final_predictions_list.append(i)
        final_reference_list.append(temp_ref_list)


    results = bleu.compute(predictions=final_predictions_list,
                          references=final_reference_list)
    print(results)
    return

In [ ]:
# Bert Score
def print_bert_score_top(uuid_theory,uuid_pred):
    """
    Reformats the data according to the input format of bertscore function.
    Calculates the bert score and prints it.

    Parameters
    ----------
    uuid_theory : Dict
        Contains the original labels
    uuid_pred : Dict
        Contains the answers predicted by the model

    Returns
    -------
    None.

    """
    final_predictions_list = []
    final_reference_list = []

    for key, value in uuid_theory.items():
      temp_ref_list= value
      temp_pred_list = uuid_pred[key]
      for i in temp_pred_list:
        final_predictions_list.append(i)
        final_reference_list.append(temp_ref_list[0])


    results = bertscore.compute(predictions=final_predictions_list, references=final_reference_list,
                                model_type= model_type_bert_score)
    # print(results)
    print("F1 Score")
    print(sum(results['f1'])/len(results['f1']))

    print("Precision Score")
    print(sum(results['precision'])/len(results['precision']))

    print("Recall Score")
    print(sum(results['recall'])/len(results['recall']))
    return

In [ ]:
#Get QA predictions for Top 10 Predictions of the Classification Model and print the Bleu and Bert Scores
predictions, _, _ = trainer.predict(tokenized_val_top10)
start_logits, end_logits = predictions
predicted_answers, theoretical_answers, examples = compute_metrics(start_logits, end_logits, tokenized_val_top10, dict_val_top10)
uuid_theory = dict()
uuid_pred = dict()
for i in range(len(theoretical_answers)):
  try:
    temp_uuid = id_uuid_mapping_top10[theoretical_answers[i]['id']]
    # print(temp_uuid)
    uuid_theory[temp_uuid] = uuid_theory.get(temp_uuid, []) + theoretical_answers[i]['answers']['text']
    uuid_pred[temp_uuid] = uuid_pred.get(temp_uuid, []) + [predicted_answers[i]['prediction_text']]
  except Exception as e:
    pass



for key, value in uuid_pred.items():
  temp_string = uuid_pred[key]
  uuid_pred[key] = list(set(temp_string))


for key, value in uuid_theory.items():
  temp_string = uuid_theory[key]
  temp_final_theory = []
  for i in temp_string:
    if len(i)> 0:
      temp_final_theory.append(i)
  if len(temp_final_theory)>0:
    uuid_theory[key] = temp_final_theory
  else:
    uuid_theory[key] = [""]

print("Bleu Score for top 10 is")
print_bleu_score_top(uuid_theory,uuid_pred )

print("Bert Score for top 10 is")
print_bert_score_top(uuid_theory,uuid_pred )

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping. If uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 6820
  Batch size = 16


100%|██████████| 6820/6820 [00:04<00:00, 1679.02it/s]


Bleu Score for top 10 is
{'bleu': 0.0019722725715446325, 'precisions': [0.2888366627497062, 0.6910286156225831, 0.6776859504132231, 0.6684406599254923], 'brevity_penalty': 0.003596723165255996, 'length_ratio': 0.15088117442643878, 'translation_length': 8510, 'reference_length': 56402}
Bert Score for top 10 is


F1 Score
0.5646756727759054
Precision Score
0.5917819998831382
Recall Score
0.5418202737742466


In [ ]:
#Get QA predictions for Top 4 Predictions of the Classification Model and print the Bleu and Bert Scores
predictions, _, _ = trainer.predict(tokenized_val_top4)
start_logits, end_logits = predictions
predicted_answers, theoretical_answers, examples = compute_metrics(start_logits, end_logits, tokenized_val_top4, dict_val_top4)
uuid_theory = dict()
uuid_pred = dict()
for i in range(len(theoretical_answers)):
  try:
    temp_uuid = id_uuid_mapping_top4[theoretical_answers[i]['id']]
    uuid_theory[temp_uuid] = uuid_theory.get(temp_uuid, []) + theoretical_answers[i]['answers']['text']
    uuid_pred[temp_uuid] = uuid_pred.get(temp_uuid, []) + [predicted_answers[i]['prediction_text']]
  except Exception as e:
    pass



for key, value in uuid_pred.items():
  temp_string = uuid_pred[key]
  uuid_pred[key] = list(set(temp_string))


for key, value in uuid_theory.items():
  temp_string = uuid_theory[key]
  temp_final_theory = []
  for i in temp_string:
    if len(i)> 0:
      temp_final_theory.append(i)
  if len(temp_final_theory)>0:
    uuid_theory[key] = temp_final_theory
  else:
    uuid_theory[key] = [""]

print("Bleu Score for top4 is")
print_bleu_score_top(uuid_theory,uuid_pred )

print("Bert Score for top4 is")
print_bert_score_top(uuid_theory,uuid_pred )

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping. If uuid, spoilerType, title, id, answers, targetParagraphId, question, example_id, context, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3147
  Batch size = 16


100%|██████████| 3147/3147 [00:01<00:00, 1624.23it/s]


Bleu Score for top4 is
{'bleu': 0.02277664871278948, 'precisions': [0.3770971517752634, 0.679097605893186, 0.6632152588555859, 0.6522284996861268], 'brevity_penalty': 0.03948024966750964, 'length_ratio': 0.23629742313188587, 'translation_length': 5126, 'reference_length': 21693}
Bert Score for top4 is


F1 Score
0.470355218307909
Precision Score
0.4908325375639755
Recall Score
0.453386812171342
